In [1]:
from torchtext import data
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import torch




In [2]:
path= '/Users/tancredicp/Desktop/sailing_app/data/ML Training Resources/'
all_files = glob.glob(path + "/*.csv")

TAGS = data.Field(unk_token = None)
TAGS.build_vocab(['0','S', 'C','G','T','B','D', 'A'])

Data = []
for filename in all_files:
    temporary_dict = {}

    dfSpecific = pd.read_csv(filename, index_col=None, header=0)
    dfSpecific.fillna(0, ianplace= True)
    
    dfSpecificY = dfSpecific['tag']
    temporary_dict['tags'] = TAGS.numericalize((np.array(dfSpecificY)))
    

    dfSpecificX = dfSpecific.drop(columns=['tag', 'tag_name', 'boat_class', 'time'])
    
    scaler = StandardScaler()
    columnsBeingStandardize = ['lat', 'lon', 'sog', 'heel', 'clew_load','pitch']
    dfSpecificX[columnsBeingStandardize] = scaler.fit_transform(dfSpecificX[columnsBeingStandardize])
    
    
    array1 = np.array(dfSpecificX[['lat', 'lon', 'sog', 'cog']])
    
    temp = torch.from_numpy(array1)
    
    temporary_dict['track'] = temp
    
    Data.append(temporary_dict)
    del(temporary_dict)
    break


In [3]:
#creating Dataset 
from torch.utils.data import Dataset
class SailingData(Dataset):
    def __init__ (self, tracks):
        self.samples = []
        
        for track in tracks:
            self.samples.append((track['track'], track['tags']))
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
            return(self.samples[idx])

In [4]:
#creating train iterator
BATCH_SIZE = 1#the automatic batch size is 1
DatasetSailing= SailingData(Data)
train_iterator = torch.utils.data.DataLoader(DatasetSailing, batch_size = BATCH_SIZE)

In [5]:
DatasetSailing[0]

(tensor([[ 0.5169, -0.5162,  1.3716, -0.3000],
         [ 0.5262, -0.5372,  1.5119, -0.3111],
         [ 0.5362, -0.5570,  1.4558, -0.2722],
         ...,
         [ 0.9956, -1.3453, -1.1912, -0.3667],
         [ 0.9956, -1.3475, -1.1459, -0.4167],
         [ 0.9958, -1.3498, -1.1697, -0.4389]], dtype=torch.float64),
 tensor([[7, 7, 7,  ..., 4, 4, 4]]))

In [106]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm1 = nn.LSTM(input_size = 4, hidden_size = 32, num_layers = 2, dropout = 0.5)
        self.fc1 = nn.Linear(32, 120)
        self.fc2 = nn.Linear(120,9)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(0)
        x = x.unsqueeze(0)
        x, _ = self.lstm1(x)
#         print(x.shape)
        x = F.dropout(x, p = 0.9)
#         print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.softmax(x)
        return x


net = Net()
net = net.double()

In [107]:
from sklearn.metrics import accuracy_score

# accuracy_score(o_lst, l_lst)

In [108]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [110]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    o_lst = []
    l_lst = []
    for i, data in enumerate(train_iterator, 0):
        inputs, labels = data
        
        #WHAT DOES THIS DO 
        optimizer.zero_grad()

#         print(labels.shape)
        for data_point, lab in zip(inputs[0], labels[0][0]):
            outputs = net(data_point)
            lab  = lab.unsqueeze(0)
            outputs = outputs.squeeze(0)
#             print('outputs', outputs.shape, 'lab', lab.shape)
            loss = criterion(outputs, lab)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        
        
            o_lst.append((outputs.argmax().item()))
            l_lst.append(lab.item())
       
#             break

       
    
    accuracy = accuracy_score(o_lst, l_lst)
    print("Epoch: ", epoch, "Loss: ",running_loss, "Accuracy: " , accuracy)
  
print('Finished Training')

Epoch:  0 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  1 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  2 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  3 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  4 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  5 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  6 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  7 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  8 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  9 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  10 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  11 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  12 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  13 Loss:  7896.825130945925 Accuracy:  0.029771841958820256
Epoch:  14 Loss:  7896.825130945925 Accuracy:  0.029771841

In [ ]:
lst = []
for i, data in enumerate(train_iterator, 0):
    inputs, labels = data
    o = net(inputs)
    lst.append(o.argmax().item())

In [209]:
inti.argmax()

tensor(4, grad_fn=<NotImplemented>)

In [173]:
TAGS.tokenize('2')

['2']

In [178]:
TAGS

In [205]:
TAGS.tokenize(torch.t[0,1])

TypeError: split() missing 1 required positional argument: 'split_size'

In [196]:
TAGS.numericalize(['A', 'S'])

tensor([[2, 7]])